# Sentiment analysis

##  Set up

In [27]:
%pip install numpy pandas nltk sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [28]:
import numpy as np 
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string


## Load dataset

In [29]:
df = pd.read_csv(r'../resources/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [30]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

## Tiền xử lý dữ liệu

### Làm sạch dữ liệu 
- Loại bỏ các thẻ HTML
- Loại bỏ khoảng trắng thừa và dấu câu 
- Loại bỏ stop words
- Chuyển đổi chữ hoa thành chữ thường 
- Chuyển đổi nhãn sang số

#### Loại bỏ thẻ HTML

In [31]:
df['review'] = df['review'].str.replace("<br />", "")
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

#### Loại bỏ khoảng trắng thừa và dấu câu

In [32]:
df['review'] = df['review'].str.translate(str.maketrans('', '', string.punctuation))

In [33]:
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

In [34]:
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.replace(r'\s+',' ', regex=True)
df['review'][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy The plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer While some may be disappointed when they realize this is not Match Point 2 Risk Addiction I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to loveThis was the most Id laughed at one of Woodys comedies in years dare I say a decade While Ive never been impressed with Scarlet Johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanThis may not be the crown jewel of his career but it was wittier than Devil Wears Prada and more interesting than Superman a great comedy to go see with friends'

#### Loại bỏ stop words

In [35]:
# Tải stop words từ nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [36]:
def remove_stop_words(text):
    words = text.split()
    filter_words = [word for word in words if word not in stop_words]
    return ' '.join(filter_words)

In [37]:
df['review'] = df['review'].apply(remove_stop_words)
df['review'][2]

'I thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy The plot simplistic dialogue witty characters likable even well bread suspected serial killer While may disappointed realize Match Point 2 Risk Addiction I thought proof Woody Allen still fully control style many us grown loveThis Id laughed one Woodys comedies years dare I say decade While Ive never impressed Scarlet Johanson managed tone sexy image jumped right average spirited young womanThis may crown jewel career wittier Devil Wears Prada interesting Superman great comedy go see friends'

#### Chuyển chữ hoa thành chữ thường 

In [38]:
df['review'] = df['review'].str.lower()
df['review'][2]

'i thought wonderful way spend time hot summer weekend sitting air conditioned theater watching lighthearted comedy the plot simplistic dialogue witty characters likable even well bread suspected serial killer while may disappointed realize match point 2 risk addiction i thought proof woody allen still fully control style many us grown lovethis id laughed one woodys comedies years dare i say decade while ive never impressed scarlet johanson managed tone sexy image jumped right average spirited young womanthis may crown jewel career wittier devil wears prada interesting superman great comedy go see friends'

#### Chuyển đổi nhãn sang số

In [39]:
def sentiment_encode(text):
    if(text == "positive"):
        return 1
    return -1

In [40]:
df['sentiment'] = df['sentiment'].apply(sentiment_encode)

### Chia tập dữ liệu

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [42]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.3, random_state=42)

In [43]:
# Chuyển đổi văn bản thành các đặc trưng số sử dụng TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000,)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [44]:
X_train_tfidf[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 47 stored elements and shape (1, 5000)>

In [45]:
sentiment = y_train.unique()
sentiment

array([-1,  1])

## Mô hình

### Decision tree

In [14]:
# Huấn luyện mô hình Decision Tree
clf = DecisionTreeClassifier(
    random_state=42, 
    max_depth= 50,
)
clf.fit(X_train_tfidf, y_train)

DecisionTreeClassifier(max_depth=50, random_state=42)

#### Kiểm tra các thuộc tính của cây 

In [15]:
print(f'Depth of the tree: {clf.get_depth()}')
print(f'Number of leaves: {clf.get_n_leaves()}')
print(f'Number of features: {clf.n_features_in_}')
print(f'Feature importances: {clf.feature_importances_}')

Depth of the tree: 50
Number of leaves: 2704
Number of features: 176077
Feature importances: [0. 0. 0. ... 0. 0. 0.]


#### Test thử trên tập train và tập test

In [16]:
y_pred_train = clf.predict(X_train_tfidf)
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.9785142857142857
Classification Report:
              precision    recall  f1-score   support

          -1       0.99      0.96      0.98     17589
           1       0.97      0.99      0.98     17411

    accuracy                           0.98     35000
   macro avg       0.98      0.98      0.98     35000
weighted avg       0.98      0.98      0.98     35000



In [17]:
# Dự đoán trên tập kiểm tra
y_pred = clf.predict(X_test_tfidf)
# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.7238666666666667
Classification Report:
              precision    recall  f1-score   support

          -1       0.73      0.71      0.72      7411
           1       0.72      0.74      0.73      7589

    accuracy                           0.72     15000
   macro avg       0.72      0.72      0.72     15000
weighted avg       0.72      0.72      0.72     15000



#### Bảng tổng kết 
| Mô hình | Tham số | Độ chính xác trên tập train | Độ chính xác | Đánh giá | 
|---------|----------|------|--------|----|
|Decision tree | default | -- | 0.7217 | --|
|Decision tree | criterion = entropy_loss | --  | 0.7202| --|
|Decision tree | criterion = log_loss | 1 | 0.7202 | Overfitting |
|Decision tree | criterion = log_loss, max_depth = 50 | 0.98 | 0.7228 | Overfitting |
|Decision tree | criterion = log_loss, max_depth = 25 | 0.89 | 0.73 | Bad |
|Decision tree | criterion = log_loss, max_depth = 37,  | 0.95 | 0.7259 | Overfitting |
|Decision tree | criterion = gini, max_depth = 37, max features = 50k | 0.95 | 0.7314 | Overfitting |
|Decision tree | criterion = gini, max_depth = 37, max leaf node = 2000 | 0.95 | 0.7337 | Overfitting |
|Decision tree | criterion = gini, max_depth = 15, max leaf node = 100 | 0.76 | 0.7467 | Bad |
|Decision tree | criterion = gini, max_depth = 15, max leaf node = 100 | 0.78 | 0.7418 | Bad |

### Logistic regression


In [33]:
from sklearn.linear_model import LogisticRegression

# Huấn luyện mô hình Logistic Regression
log_regr = LogisticRegression(
    random_state=42,
)
log_regr.fit(X_train_tfidf, y_train)


LogisticRegression(random_state=42)

In [34]:
log_regr.n_iter_

array([79])

In [35]:
y_pred_train = log_regr.predict(X_train_tfidf)
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)
print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.9362
Classification Report:
              precision    recall  f1-score   support

          -1       0.94      0.93      0.94     17589
           1       0.93      0.94      0.94     17411

    accuracy                           0.94     35000
   macro avg       0.94      0.94      0.94     35000
weighted avg       0.94      0.94      0.94     35000



In [36]:

# Dự đoán trên tập kiểm tra
y_pred = log_regr.predict(X_test_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)

Accuracy: 0.8975
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.89      0.90      7411
           1       0.89      0.91      0.90      7589

    accuracy                           0.90     15000
   macro avg       0.90      0.90      0.90     15000
weighted avg       0.90      0.90      0.90     15000



### Random forest

In [21]:
# Import các thư viện cần thiết
from sklearn.ensemble import RandomForestClassifier

# Huấn luyện mô hình Random Forest
rf_clf = RandomForestClassifier(
    random_state=42,
)
rf_clf.fit(X_train_tfidf, y_train)


RandomForestClassifier(random_state=42)

In [26]:
# Dự đoán trên tập kiểm tra
y_pred_train = rf_clf.predict(X_train_tfidf)

# Đánh giá mô hình
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     17589
           1       1.00      1.00      1.00     17411

    accuracy                           1.00     35000
   macro avg       1.00      1.00      1.00     35000
weighted avg       1.00      1.00      1.00     35000



In [27]:
# Dự đoán trên tập kiểm tra
y_pred = rf_clf.predict(X_test_tfidf)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)



Accuracy: 0.841
Classification Report:
              precision    recall  f1-score   support

          -1       0.83      0.85      0.84      7411
           1       0.85      0.83      0.84      7589

    accuracy                           0.84     15000
   macro avg       0.84      0.84      0.84     15000
weighted avg       0.84      0.84      0.84     15000



depth = 15 --> 0.8285 \
depth = 37 --> 0.84

### SVM

In [47]:
from sklearn.decomposition import TruncatedSVD
# 5. Giảm chiều với Truncated SVD
svd = TruncatedSVD(n_components=3000, random_state=42)
X_train_reduced = svd.fit_transform(X_train_tfidf)
X_test_reduced = svd.transform(X_test_tfidf)

In [48]:
from sklearn.svm import SVC

In [59]:
svm_classifier = SVC(
    C=100, 
    gamma=0.1, 
    kernel='poly', 
    degree=3, 
    coef0=0.1, 
    random_state=42
)

In [60]:
svm_classifier.fit(X_train_reduced,y_train)

SVC(C=100, coef0=0.1, gamma=0.1, kernel='poly', random_state=42)

In [61]:

print(svm_classifier)

SVC(C=100, coef0=0.1, gamma=0.1, kernel='poly', random_state=42)


In [62]:
# Dự đoán trên tập kiểm tra
y_pred_train = svm_classifier.predict(X_train_reduced)

# Đánh giá mô hình
accuracy_train = accuracy_score(y_train, y_pred_train)
report_train = classification_report(y_train, y_pred_train)

print(f'Accuracy: {accuracy_train}')
print('Classification Report:')
print(report_train)

Accuracy: 0.9477714285714286
Classification Report:
              precision    recall  f1-score   support

          -1       0.95      0.94      0.95     17589
           1       0.94      0.95      0.95     17411

    accuracy                           0.95     35000
   macro avg       0.95      0.95      0.95     35000
weighted avg       0.95      0.95      0.95     35000



In [63]:
# Dự đoán trên tập kiểm tra
y_pred = svm_classifier.predict(X_test_reduced)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8902
Classification Report:
              precision    recall  f1-score   support

          -1       0.90      0.88      0.89      7411
           1       0.88      0.90      0.89      7589

    accuracy                           0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000

